# Start
these libraries are neccesary to install to use this code. specific versions must be installed. just run the code below for the first time when you will run this file.

In [1]:
# !pip install pillow==10.4.0
# !pip install tensorflow==2.8.3
# !pip install ultralytics==8.3.23
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install opencv-python==4.10.0.84
# !pip install numpy==1.26.4


In [18]:
import os
import time
from datetime import datetime
import json
import random
import tkinter as tk
from PIL import Image, ImageTk
import matplotlib.pyplot as plt
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.utils import load_img,img_to_array
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input, decode_predictions
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Input,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import RandomCrop, RandomFlip, RandomRotation, RandomContrast
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.regularizers import l2
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from tensorflow.keras import backend as K
import torch
from ultralytics import YOLO
from tkinter import simpledialog, filedialog

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

# Clear last session to feee up space before new
K.clear_session()

# if GPU is available this code will state 1
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# image height and width for input into Siamese Network with MobileNetV3 as a base 
IMG_HEIGHT = 300
IMG_WIDTH = 300


Num GPUs Available:  1


In [3]:
# load Siamese network model file
def load_model(model_name):
    model = tf.keras.models.load_model(model_name)
    return model

# Preprocessing function to resize and normalize image
def preprocess_image(image):
    # resize image to match height and width specified earlier 
    image_resized = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
    
    # Normalize values of pixels to be around 0.0-1.0
    image_resized = image_resized / 255.0  
    
    return image_resized

In [4]:
# siamese network prediction function  
def predict(id_images, img_to_evaluate, model, threshold = 0.65):
    '''
    id_images_batch - batch of saved id images, should be numpy array of shape (x, 300, 300, 3) where x is number of saved images 
    or path for image which is saved in the memory and should be compared to, in this case it is string path
    img_to_evaluate - image which will be compared to img_base which is saved in the memory
    model - AI model file which contains trained weights for siamese network
    threshold - optional value which we can state for siamese networks confidence score from 0.50 till 0.99, which means 
    that results will be shown only if confidence score for similarity is more than threshold
    '''
    
    # preprocess images
    if type(id_images) == str:
        img_base = np.array(preprocess_image(id_images))
        img_base = np.expand_dims(img_base, axis=0)  # Add batch dimension, shape becomes (1, 300, 300, 3)
    elif isinstance(id_images, np.ndarray):
        img_base = id_images

    
    img_to_evaluate = np.array(preprocess_image(img_to_evaluate))

    # expand dimensions of images to add batch dimension 
    
    img_to_evaluate = np.expand_dims(img_to_evaluate, axis=0)  # Add batch dimension, shape becomes (1, 300, 300, 3)
    
    img_to_evaluate = np.tile(img_to_evaluate, (len(img_base), 1, 1, 1))
    
    # predict the similarity between the two images,
    # returns tensor with two values like this [0.37, 0.63] which states how similar are images
    # 0.0 no similarity , 1.0 full similarity
    preds = model.predict((img_base,img_to_evaluate))
    predicted_idx = np.argmax(preds[:, 1])

    if preds[predicted_idx][1] > threshold:
        return predicted_idx, preds[predicted_idx][1]
    else:
        return 999,999
    # if the model is sure that shown images are more similar than threshold it will output the 1, if not 0
    # if predictions are less than a threshold it will output 2   
    
    # if preds[0][0] > threshold:
    #     predicted_idx = 0
    # elif preds[0][1] > threshold:
    #     predicted_idx = 1
    # else:
    #     predicted_idx = 2
    # predicted_idx = np.argmax(preds)
    # funciton also returns the predictions list
    

In [5]:
# load the siamese network model
model_name = f'MobileNetV3_1024_siamese.h5'
model = load_model(model_name)
# load the YOLO model 
yolo_model = torch.hub.load('ultralytics/yolov5',"custom", path=r'.\yolov5\runs\train\exp6\weights/best.pt',force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\Saba/.cache\torch\hub\master.zip
YOLOv5  2024-11-23 Python-3.9.0 torch-2.5.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [6]:
# function returns YOLO model object detection results 
def yolo(yolo_model,frame):
    results = yolo_model(frame)
    return results

# unpack the coordinates of bounding boxes and confidence score for ID class
def get_results_of_yolo(frame_predictions):
    x1 = int(frame_predictions.xyxy[0][0][0])
    y1 = int(frame_predictions.xyxy[0][0][1])
    x2 = int(frame_predictions.xyxy[0][0][2])
    y2 = int(frame_predictions.xyxy[0][0][3])
    
    confidence = frame_predictions.xyxy[0][0][4]
    
    return x1,y1,x2,y2,confidence

# crop the id card from the frame for it to be saved or passed to siamese network for evaluating
def crop_id_card(frame_predictions, frame):
    x1,y1,x2,y2,confidence = get_results_of_yolo(frame_predictions)
    # x1 (pixels)  y1 (pixels)  x2 (pixels)  y2 (pixels)   confidence   class
    cropped_image = frame[y1:y2, x1:x2]
    
    return cropped_image

In [7]:
# file path for id_examples
id_examples_filepath = './id_examples/'
def retrieve_saved_ids():
    saved_ids_labels = []
    saved_ids_images = []
    for id_img in os.listdir(id_examples_filepath):
        id_image_uncropped = cv2.imread(id_examples_filepath+id_img)
        bounding_box_prediction = yolo(yolo_model,id_image_uncropped)
        
        # make siamese network work only in the case of finding id card shown
        if bounding_box_prediction.xyxy[0].shape != (0,6):
            # get coordinates of bounding box
            x1,y1,x2,y2,confidence_score = get_results_of_yolo(bounding_box_prediction)
            id_image_cropped = crop_id_card(bounding_box_prediction, id_image_uncropped)
            saved_ids_images.append(id_image_cropped)
        else: 
            saved_ids_images.append(id_image_uncropped)
            print("uncropped version added")
        saved_ids_labels.append(id_img)

    id_img_batch = np.array([preprocess_image(image_array) for image_array in saved_ids_images])
    if id_img_batch.size == 0:
        id_img_batch = np.ones((1, 300, 300, 3), dtype=np.uint8) * 255
        saved_ids_labels.append("Nothing")
    return id_img_batch, saved_ids_labels



In [8]:
def add_new_id_to_database(frame, new_id_name):
    # Create a directory to store new ID images if it doesn’t exist
    os.makedirs("./id_examples/", exist_ok=True)

    # Create a unique file name with ID type and current timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    file_path = f"./id_examples/{new_id_name}_{timestamp}.jpg"

    # Save the image to the specified path
    cv2.imwrite(file_path, frame)

In [9]:
# def is_still_frame(current_frame, previous_frame, threshold=50000):
#     # Calculate the absolute difference between frames
#     diff = cv2.absdiff(previous_frame, current_frame)
#     # Convert to grayscale and compute the sum of differences
#     gray_diff = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
#     non_zero_count = np.sum(gray_diff)
#     return non_zero_count < threshold

# def is_sharp_frame(frame, threshold=50):
#     # Calculate the Laplacian to assess sharpness (higher variance = sharper)
#     gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     variance = cv2.Laplacian(gray_frame, cv2.CV_64F).var()
#     return variance > threshold

def ask_user_for_id_type():
    root = tk.Tk()
    root.withdraw()
    new_id_type = simpledialog.askstring("Input", "Enter new ID type name:")
    root.destroy()
    return new_id_type
    
# Create a button that prints the input field's text when clicked
def on_button_click(id_image):
    new_id_name = input_entry.get()  # Get text from entry field
    print(new_id_name + "es daarqva")
    if new_id_name != None:
        add_new_id_to_database(id_image, new_id_name)
        id_images_batch, id_labels = retrieve_saved_ids()
        input_entry.delete(0, tk.END)  # Clear the entry field after clicking
    else:
        return None

def create_window_submit_window():
    window_submit_photo = tk.TK()
    window_submit_photo.title("ID submission")
    window.geometry("1200x800")
    
    video_label_submit = tk.Label(window_submit_photo)
    video_label_submit.pack()
    
    input_entry = tk.Entry(window, width=30)
    input_entry.pack()

    button = tk.Button(window, text="Submit", command=on_button_click)
    button.pack()
    
def check_for_new_id_type(frame,confidence_score):
    global frame_counter
    if confidence_score > confidence_score_threshold:
        if frame_counter < 20:
            frame_counter += 1
            return False
        else:
            new_id_name = ask_user_for_id_type()
            if new_id_name != None:
                add_new_id_to_database(frame, new_id_name)
                frame_counter = 0
                return True
            else:
                return False
    return False

In [10]:
def find_mrz_area(image):
    """
    Find the area where the MRZ is located in an ID card image.
    
    Parameters:
    image (numpy.ndarray): The input ID card image.
    
    Returns:
    tuple: The bounding box (x, y, w, h) of the MRZ area.
    """

    # initialize a rectangular and square structuring kernel (this size is dependent on the ID-Card size)
    rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (21, 5))
    sqKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 31))

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply GaussianBlur to reduce noise and improve contour detection
    gray = cv2.GaussianBlur(gray, (3, 3), 0)
    # cv2.imshow('Gray', gray)

    # Apply blackhat morphological operation to enhance the text
    blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, rectKernel)
    # cv2.imshow('Blackhat', blackhat)

    # apply a closing operation using the rectangular kernel to close
	# gaps in between letters -- then apply Otsu's thresholding method
    blackhat_closed = cv2.morphologyEx(blackhat, cv2.MORPH_CLOSE, rectKernel)
    # cv2.imshow('blackhat_closed', blackhat_closed)
    thresh = cv2.threshold(blackhat_closed, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    # cv2.imshow('Thresh', thresh)

    # perform another closing operation, this time using the square
	# kernel to close gaps between lines of the MRZ
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, sqKernel)
    # cv2.imshow('Thresh_closed', thresh)

    # Convert the thresholded image to a color image to plot the contours in color
    thresh_color = cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR)
    
	# find contours in the thresholded image and sort them by their
	# size
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
		cv2.CHAIN_APPROX_SIMPLE)
    # Handle different versions of OpenCV
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    # Sort contours by area in descending order    
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

    # Initialize ROI as None
    roi = None

	# loop over the contours
    for i, c in enumerate(cnts):
		# compute the bounding box of the contour and use the contour to
		# compute the aspect ratio and coverage ratio of the bounding box
		# width to the width of the image
        (x, y, w, h) = cv2.boundingRect(c)
         # Draw the bounding box on the color thresholded image
        color = (0, 255, 0) if i == 0 else (0, 0, 255)  # Green for the first (largest) contour, red for others
        cv2.rectangle(thresh_color, (x, y), (x + w, y + h), color, 2)
        # cv2.imshow('Thresholded Image with Contours', thresh_color)

        ar = w / float(h)
        crWidth = w / float(gray.shape[1])
		# check to see if the aspect ratio and coverage width are within
		# acceptable criteria
        if ar > 4 and crWidth > 0.75:
			# pad the bounding box to have some space for later reading
            pad = 5
            x = x - pad
            y = y - pad
            w = w + (pad * 2)
            h = h + (pad * 2)

			# extract the ROI from the image and draw a bounding box
			# surrounding the MRZ
            roi = image[y:y + h, x:x + w]

            break

    return (x, y, w, h), roi if roi is not None else None

In [13]:

confidence_score_threshold = 0.7

# Open video capture (0 is the default camera)
video_path = "./passport_video.mp4"
cap = cv2.VideoCapture(0)
id_images_batch, id_labels = retrieve_saved_ids()
frame_counter = 0  # Use local variable instead of global

fps = cap.get(cv2.CAP_PROP_FPS)
if fps == 0:  # Avoid division by zero
    fps = 30  # Default to 30 FPS if unable to retrieve
frame_delay = int(1000 / fps)  # Calculate frame delay in milliseconds
print(frame_delay)


# Initialize the Tkinter window
window = tk.Tk()
window.title("ID Detection")
window.geometry("1200x800")


# Create a label to display the video feed
video_label = tk.Label(window)
video_label.pack()

# Function to update frame in Tkinter window
def update_frame():
    global frame_counter, id_images_batch, id_labels

    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        window.after(10, update_frame)  # Schedule next frame update
        return

    
    frame = cv2.resize(frame, (800, 600))
    
    # # Predict bounding box coordinates
    # bounding_box_prediction = yolo(yolo_model, frame)

    # # Make siamese network work only if ID card is found
    # if bounding_box_prediction.xyxy[0].shape != (0, 6):
    #     # Get coordinates of bounding box
    #     x1, y1, x2, y2, confidence_score = get_results_of_yolo(bounding_box_prediction)

    #     # Crop the ID card
    #     cropped_id_card = crop_id_card(bounding_box_prediction, frame)

    #     # Check if a new ID card is detected

    
    #     # Predict with siamese network
    #     prediction_idx, confidence_score_siamese = predict(id_images_batch, cropped_id_card, model, threshold=0.95)

    #     if prediction_idx == 999:
    #         prediction_name = "not found"
    #         if check_for_new_id_type(frame,confidence_score) == True:
    #             id_images_batch, id_labels = retrieve_saved_ids()
                
    #     else:
    #         prediction_name = id_labels[prediction_idx]

    #     cv2.putText(frame, f'id type: {prediction_name}, {confidence_score_siamese:.2f}', (10, 30),
    #                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
    #     if confidence_score > 0.45:
    #         # Draw bounding box on the frame
    #         cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    #         text = f'{confidence_score:.2f}'
    #         text_position = (x1, y1 - 10)
    #         cv2.putText(frame, text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    mrz_area, roi = find_mrz_area(frame)

    if mrz_area is not None:
        x, y, w, h = mrz_area
        print(mrz_area)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
        # cv2.imshow('MRZ Area', frame)
        if roi is not None:
            # print(roi)
            if roi.shape[0]>0 and roi.shape[1]>0:
                cv2.imshow('MRZ ROI', roi)
    else:
        print("MRZ area not found.")
    # Convert frame to RGB and update the Tkinter label
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(frame_rgb)
    imgtk = ImageTk.PhotoImage(image=img)
    video_label.imgtk = imgtk
    video_label.configure(image=imgtk)

    # Schedule the next frame update
    video_label.after(frame_delay, update_frame)

# Start updating frames
update_frame()

# Define a function to handle window close
def on_closing():
    cap.release()  # Release the camera
    window.destroy()  # Close the Tkinter window

# Bind the window close event
window.protocol("WM_DELETE_WINDOW", on_closing)

# Start the Tkinter main loop
window.mainloop()


33
(416, 135, 4, 1)
(331, 338, 1, 1)
(355, 111, 1, 2)
(238, 332, 3, 1)
(339, 300, 1, 1)
(0, 306, 8, 1)
(316, 142, 2, 1)
(169, 180, 4, 1)
(454, 303, 1, 1)
(89, 24, 1, 1)
(335, 145, 1, 2)
(359, 54, 2, 1)
(314, 176, 1, 1)
(448, 280, 1, 1)
(68, 131, 1, 1)
(68, 131, 1, 1)
(564, 310, 2, 1)
(790, 243, 10, 1)
(539, 342, 1, 1)
(695, 461, 1, 1)
(157, 179, 3, 1)
(365, 369, 2, 1)
(135, 175, 5, 1)
(346, 385, 4, 1)
(298, 354, 1, 1)
(414, 290, 2, 1)
(443, 249, 1, 1)
(220, 292, 2, 1)
(464, 181, 9, 1)
(466, 163, 1, 1)
(596, 363, 2, 2)
(323, 476, 1, 1)
(359, 474, 5, 1)
(106, 495, 3, 2)
(755, 241, 8, 1)
(59, 96, 2, 1)
(365, 284, 1, 1)
(315, 19, 5, 1)
(605, 66, 4, 1)
(628, 0, 1, 12)
(551, 0, 1, 9)
(120, 378, 644, 85)
(121, 380, 644, 84)
(121, 397, 650, 74)
(120, 400, 653, 72)
(115, 401, 652, 70)
(113, 400, 652, 69)
(110, 398, 652, 69)
(107, 397, 652, 68)
(101, 388, 648, 70)
(100, 387, 648, 69)
(100, 384, 647, 71)
(404, 44, 1, 1)
(409, 54, 1, 1)
(431, 54, 1, 2)
(66, 375, 646, 73)
(64, 375, 647, 72)
(62, 37

In [30]:
image = cv2.imread('./spanish_id_front.jpg')

def find_face(image):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)
    return image
image = find_face(image)
image = cv2.resize(image,(960,540))


In [106]:
image = cv2.imread('./spanish_id_front.jpg')

rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 47))

# Apply blackhat morphology
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, rectKernel)

# Threshold and find contours
_, thresh = cv2.threshold(blackhat, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter contours for signature area
height, width, dim = image.shape
for c in contours:
    x, y, w, h = cv2.boundingRect(c)
    if w > 200 and h > 100:
        aspect_ratio = w / float(h)

        if 1.5 < aspect_ratio < 4.5: 
            if 2 < ((w*h) * 100)/(height * width) < 5: # Signature is often wide
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            
image = cv2.resize(image,(960,540))
cv2.imshow("lala", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

137 406
1152 2048
2.3575676812065973


2.0
1.5285714285714285


4.1551246537396125